#### 1. Import pandas library

In [43]:
import pandas as pd
import numpy as np

#### 2. Import users table:

In [44]:
users = pd.read_csv("users_table.csv")

# 3. Rename Id column to userId

In [42]:
users = users.rename(columns={'Id':'userId'})

#### 4. Import posts table:

In [46]:
posts = pd.read_csv('posts_table.csv')
posts

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,48321,2,NaN,2013-01-23 09:00:01,0,NaN,<p>you can use the matlab codes for svm and co...,19966.0,2013-01-23 09:00:01,NaN,...,NaN,0,NaN,NaN,NaN,NaN,45118.0,NaN,NaN,NaN
39996,48322,2,NaN,2013-01-23 09:09:34,3,NaN,"<p>I use <a href=""http://www.gnu.org/software/...",892.0,2013-01-23 13:13:30,NaN,...,NaN,2,NaN,892.0,2013-01-23 13:13:30,NaN,48311.0,NaN,NaN,NaN
39997,48323,2,NaN,2013-01-23 09:16:44,1,NaN,"<p>If I understand your question correctly, yo...",2020.0,2013-01-23 09:16:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48247.0,NaN,NaN,NaN
39998,48324,2,NaN,2013-01-23 09:36:07,3,NaN,"<p>Doesn't really help you with your question,...",19914.0,2013-01-23 09:36:07,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48297.0,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [47]:
posts = posts.rename(columns={'Id':'postId', 'OwnerUserId':'userId'})
posts

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,48321,2,NaN,2013-01-23 09:00:01,0,NaN,<p>you can use the matlab codes for svm and co...,19966.0,2013-01-23 09:00:01,NaN,...,NaN,0,NaN,NaN,NaN,NaN,45118.0,NaN,NaN,NaN
39996,48322,2,NaN,2013-01-23 09:09:34,3,NaN,"<p>I use <a href=""http://www.gnu.org/software/...",892.0,2013-01-23 13:13:30,NaN,...,NaN,2,NaN,892.0,2013-01-23 13:13:30,NaN,48311.0,NaN,NaN,NaN
39997,48323,2,NaN,2013-01-23 09:16:44,1,NaN,"<p>If I understand your question correctly, yo...",2020.0,2013-01-23 09:16:44,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48247.0,NaN,NaN,NaN
39998,48324,2,NaN,2013-01-23 09:36:07,3,NaN,"<p>Doesn't really help you with your question,...",19914.0,2013-01-23 09:36:07,NaN,...,NaN,0,NaN,NaN,NaN,NaN,48297.0,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [51]:
users = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
print(users)
posts =  posts[['postId', 'Score','userId','ViewCount','CommentCount']]
posts.head(10)
# ahhhhhhhhhhh


KeyError: "['userId'] not in index"

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [52]:
merge = users.merge(posts, left_on='userId', right_on='postId')
merge.head()

KeyError: 'userId'

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [53]:
merge.isnull()
merge.isnull().sum()

NameError: name 'merge' is not defined

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [ ]:
#sorry it is not owrking  :(((

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 